In [2]:
# 参数管理: 我们可以通过一些特殊的操作对每层当中的网络当中的参数进行操作
# 包括初始化, 访问参数: 用于调试, 诊断和可视化. 以及生成共享层


In [3]:
# 我们先来看一些具有单隐藏层的多层感知机.

import torch
from torch import nn

# create Sequential: a sigual MLP (multilayer perceptron )
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
output = net(X)

output

tensor([[0.3476],
        [0.3987]], grad_fn=<AddmmBackward0>)

In [4]:
# 参数访问:
# 我们可以将Sequential看作一个列表容器, 我们可以通过索引看到其中的各个层的值
print(net[0]) # 取出第一层
# Linear(in_features=4, out_features=8, bias=True)

# 通过state_dict()返回这个层的参数. 
print(net[2].state_dict())
# 返回OrderDict这个数据类型.


Linear(in_features=4, out_features=8, bias=True)
OrderedDict([('weight', tensor([[ 0.2835,  0.3174,  0.2537,  0.2204,  0.3396,  0.2397,  0.0625, -0.2372]])), ('bias', tensor([0.2118]))])


In [5]:
# 目标参数, 
# 注意, 其实每个参数都表示为参数类的一个实例, 在torch中就是Parameter类
print(type(net[2].bias))
print(net[2].bias) # 打印bias的属性, 其中包括data, grad两个方面
dir(net[2].bias) # 查看bias的attribute
# 通过data取出数据: 
print(net[2].bias.data) # 就是单独的data
# 通过grad取出梯度
print(net[2].bias.grad) # None, 因为我们还没backward所以还没有计算梯度. 

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2118], requires_grad=True)
tensor([0.2118])
None


In [7]:
# 访问梯度
net[2].weight.grad == None # True
# 因为我们还没有进行backward所以还没有计算梯度, 所以grad为None


True

In [18]:
# 一次性访问所有参数:
print(net.named_parameters()) # <generator object Module.named_parameters at 0x7fabcee9d7e0>
# 是一个生成器对象, 通过yield返回数值. 


# 访问net[0]的所有参数:
print(*[(name, parm) for name, parm in net[0].named_parameters()])

print("="*50)
# 访问net中的所有层, 所有参数:
print(*[(name, parm) for name, parm in net.named_parameters()]) # 返回net每层中参数 
# *进行解包. 


<generator object Module.named_parameters at 0x7fabcee9e960>
('weight', Parameter containing:
tensor([[-0.3674,  0.2823, -0.4906, -0.1248],
        [-0.0695,  0.3432,  0.2848,  0.2977],
        [-0.4307, -0.3135, -0.3763, -0.2644],
        [ 0.0667, -0.1957,  0.3242, -0.4479],
        [ 0.0710, -0.2038,  0.3748,  0.2906],
        [ 0.4225,  0.2881, -0.3994, -0.1932],
        [ 0.4537,  0.3247, -0.0122, -0.3674],
        [ 0.3074,  0.4083, -0.1229, -0.1153]], requires_grad=True)) ('bias', Parameter containing:
tensor([ 0.0159, -0.4975,  0.4451, -0.0991,  0.0323,  0.1748, -0.0334, -0.4812],
       requires_grad=True))
('0.weight', Parameter containing:
tensor([[-0.3674,  0.2823, -0.4906, -0.1248],
        [-0.0695,  0.3432,  0.2848,  0.2977],
        [-0.4307, -0.3135, -0.3763, -0.2644],
        [ 0.0667, -0.1957,  0.3242, -0.4479],
        [ 0.0710, -0.2038,  0.3748,  0.2906],
        [ 0.4225,  0.2881, -0.3994, -0.1932],
        [ 0.4537,  0.3247, -0.0122, -0.3674],
        [ 0.3074,  

In [20]:
# 因为state_dict()返回的本质是一个字典形式: 所以我们还可以通过key的形式进行访问参数
net.state_dict()['2.bias'].data # 通过key来查找

tensor([0.2118])

In [24]:

# 现在让我们看看通过add_module添加网络层, 这样我们可以指定每层的名称

# 我们先定义black1:


def black1() -> nn.Sequential:
    return nn.Sequential(
        nn.Linear(4, 8), nn.ReLU(),
        nn.Linear(8, 4), nn.ReLU()
    )

# 然后定义black2: 
def black2() -> nn.Module:
    net = nn.Sequential() # definite a container 
    
    for i in range(4):
        # 嵌套
        net.add_module(f"black: {i}", black1())
        # 和我们直接在Sequential中定义的唯一区别, 就是这里我们可以执行模型的名字
    return net

rgnet = nn.Sequential(black2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0532],
        [0.0532]], grad_fn=<AddmmBackward0>)

In [25]:
# 打印网络结构:
print(rgnet)

Sequential(
  (0): Sequential(
    (black: 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (black: 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (black: 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (black: 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)
